<h1 style="font-size:2em; font-family:sans-serif">
  Ejercicio QKD
</h1>

<sub style="color: #888;">Adrian Rodriguez, 2025</sub>

In [1]:
import random
import pandas as pd

ROUNDS = 15

<div style=" border-left: 5px solid #ff9800; padding: 1em; margin-bottom: 1em;">
  <b>¿Qué hace esta función?</b><br>
  <ul style="margin: 0.3em 0 0 1.3em;">
    <li>Simula el protocolo BB84 de distribución cuántica de claves.</li>
    <li>Alice y Bob eligen bits y bases al azar en cada ronda.</li>
    <li>Eve (espía) puede interceptar y causar errores, según <code>interception_rate</code>.</li>
    <li>Solo se conservan los bits donde las bases coinciden para la clave final.</li>
    <li>Imprime la tabla de resultados, la cantidad de bits útiles y la tasa de error (QBER).</li>
  </ul>
</div>

In [2]:
def simulate_bb84(interception_rate: float = 0.0):
    rows = []
    for _ in range(ROUNDS):
        alice_bit = random.randint(0, 1)
        alice_basis = random.randint(0, 1)  # 0 = rectilinear, 1 = diagonal

        eve_intercepts = random.random() < interception_rate
        if eve_intercepts:
            eve_basis = random.randint(0, 1)
            if eve_basis == alice_basis:
                eve_bit = alice_bit
            else:
                eve_bit = random.randint(0, 1)
            photon_bit = eve_bit
            photon_basis = eve_basis
        else:
            photon_bit = alice_bit
            photon_basis = alice_basis

        bob_basis = random.randint(0, 1)
        if bob_basis == photon_basis:
            bob_bit = photon_bit
        else:
            bob_bit = random.randint(0, 1)

        bases_match = alice_basis == bob_basis
        use_for_key = bases_match

        rows.append(
            {
                "Alice Bit": alice_bit,
                "Alice Basis": "Rectilinear" if alice_basis == 0 else "Diagonal",
                "Bob Basis": "Rectilinear" if bob_basis == 0 else "Diagonal",
                "Bob Bit": bob_bit,
                "Bases Match?": bases_match,
                "Use for Key?": "Yes" if use_for_key else "No",
            }
        )
    df = pd.DataFrame(rows)

    sifted = df[df["Bases Match?"]]
    if not sifted.empty:
        errors = (sifted["Alice Bit"] != sifted["Bob Bit"]).sum()
        error_rate = 100 * errors / len(sifted)
    else:
        error_rate = 0.0
    print(df)
    print(f"\nKey bits: {len(sifted)}")
    print(f"Percentage of rounds kept: {100 * len(sifted) / ROUNDS:.1f}%")
    print(f"QBER (error rate in sifted key): {error_rate:.1f}%")


In [3]:
simulate_bb84(0)

    Alice Bit  Alice Basis    Bob Basis  Bob Bit  Bases Match? Use for Key?
0           1     Diagonal  Rectilinear        1         False           No
1           0     Diagonal     Diagonal        0          True          Yes
2           0  Rectilinear  Rectilinear        0          True          Yes
3           1     Diagonal     Diagonal        1          True          Yes
4           0  Rectilinear  Rectilinear        0          True          Yes
5           0  Rectilinear  Rectilinear        0          True          Yes
6           1  Rectilinear  Rectilinear        1          True          Yes
7           1     Diagonal  Rectilinear        0         False           No
8           1     Diagonal     Diagonal        1          True          Yes
9           0  Rectilinear  Rectilinear        0          True          Yes
10          1  Rectilinear     Diagonal        1         False           No
11          0     Diagonal  Rectilinear        1         False           No
12          

### A. ¿Cuántos bits finales obtuvieron de la clave?

En la simulación sin intervención, se obtuvieron 9 bits finales útiles para la clave compartida.

### B. ¿Qué porcentaje representa respecto al total?

Estos 9 bits representan el 60% del total de 15 rondas realizadas.

*Nota: Dado que las siguientes preguntas involucran a Eve, se volvió a correr la simulación con una tasa de intercepción del 30%.*

In [5]:
simulate_bb84(0.3)

    Alice Bit  Alice Basis    Bob Basis  Bob Bit  Bases Match? Use for Key?
0           0     Diagonal  Rectilinear        1         False           No
1           0  Rectilinear     Diagonal        0         False           No
2           1  Rectilinear  Rectilinear        0          True          Yes
3           0  Rectilinear  Rectilinear        0          True          Yes
4           0  Rectilinear     Diagonal        0         False           No
5           0     Diagonal     Diagonal        0          True          Yes
6           1     Diagonal  Rectilinear        1         False           No
7           1     Diagonal  Rectilinear        1         False           No
8           0     Diagonal  Rectilinear        1         False           No
9           0  Rectilinear  Rectilinear        0          True          Yes
10          1  Rectilinear  Rectilinear        1          True          Yes
11          1  Rectilinear     Diagonal        1         False           No
12          

### C. ¿Qué pasaría si Eve interceptara los fotones y usara bases incorrectas?

Cuando Eve intercepta el 30 % de los fotones, la tasa de error en los bits filtrados (QBER) sube a 14,3 %. Esa cifra supera el ruido típico de un canal honesto (≈ 1 %–5 %) y se acerca o sobrepasa el umbral operativo (~11 %), por lo que la clave se volvería insegura.

### D. ¿Cómo se puede detectar su presencia?

La presencia de Eve se detecta cuando, a pesar de que Alice y Bob **usaron la misma base**, sus bits no coinciden en una proporción mayor a la esperada.

Al comparar públicamente una fracción de los bits generados, si se observan discrepancias (errores) por encima del umbral aceptable se concluye que alguien ha interferido con la transmisión. En esta simulación, la QBER de 16.7% revela claramente la intervención de Eve.


### E. ¿Qué ventajas y desventajas tiene este protocolo frente a otros cifrados tradicionales?

| **Ventajas (para el intercambio de claves)**                                           | **Desventajas**                                                                          |
|---------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------|
| Ofrece seguridad basada en la física: cualquier medición no autorizada introduce errores detectables. | Requiere hardware cuántico especializado y canales ópticos de alta calidad.              |
| Permite detectar intrusos en tiempo real midiendo el QBER.                            | Limitado a distancias de unos cientos de kilómetros sin repetidores cuánticos; la velocidad de bits útil es baja. |
| No depende de conjeturas matemáticas; es robusto ante futuros computadores cuánticos. | El canal público de autenticación sigue necesitando un esquema clásico seguro y claves iniciales. |